In [1]:
from __future__ import annotations

import os
import pickle

import numpy as np
from qtpyt.block_tridiag import greenfunction
from qtpyt.hybridization import Hybridization
from qtpyt.projector import ProjectedGreenFunction
# from scipy.linalg import eigvalsh

new_path = '/capstor/scratch/cscs/ajayaraj/qtpyt-tests/pentadienyl'
os.chdir(new_path)

/users/ajayaraj/miniconda3/envs/qtpyt/lib/python3.9/site-packages/numba/cpython/hashing.py:524: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


In [2]:
# Data paths
data_folder = f"./output/lowdin"

# Load data
index_active_region = np.load(f"{data_folder}/index_active_region.npy")
self_energy = np.load(f"{data_folder}/self_energy.npy", allow_pickle=True)
with open(f"{data_folder}/hs_list_ii.pkl", "rb") as f:
    hs_list_ii = pickle.load(f)
with open(f"{data_folder}/hs_list_ij.pkl", "rb") as f:
    hs_list_ij = pickle.load(f)

# Parameters
eta = 1e-3

# Green's Function Setup
gf = greenfunction.GreenFunction(
    hs_list_ii,
    hs_list_ij,
    [(0, self_energy[0]), (len(hs_list_ii) - 1, self_energy[1])],
    solver="dyson",
    eta=eta,
)
gfp = ProjectedGreenFunction(gf, index_active_region)

hyb = Hybridization(gfp)

In [4]:
static_hybr = hyb.retarded(0.0).real

In [6]:
trace_static_hybr = np.trace(static_hybr)
print(f"Trace of static hybridization: {trace_static_hybr:.6f}")

Trace of static hybridization: -2.337927


In [9]:
hyb.H.diagonal()

array([-4.70714113+0.j, -0.58256369+0.j,  0.33240253+0.j,  0.49577817+0.j,
        0.35935208+0.j, -0.27296628+0.j, -4.24634417+0.j])

In [10]:
static_hybr = hyb.retarded(1000.0).real
trace_static_hybr = np.trace(static_hybr)
print(f"Trace of static hybridization: {trace_static_hybr:.6f}")

Trace of static hybridization: 0.104740
